In [ ]:
# !find ./log/* -type f -not -name "stepNum*" -exec rm -f {} \;
# clean the json format log and it's corresponding csv file

In [ ]:
import pandas
import matplotlib.pyplot as plt
import json
import matplotlib
# 设置字体为 Times New Roman
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Read the data
data = pandas.read_csv('./norm/norm10.csv', header=None)
data

In [ ]:
class Log():
    def __init__(self, json_log_path):
        self.json_log_path = json_log_path
        self.param = json.load(open(self.json_log_path, 'r'))
        self.param_names = list(self.param.keys())
    def __getitem__(self, key):
        assert key in self.param.keys(), "key not in param"
        return self.param[key]
    
    def get_data(self,nrows=None):
        if nrows is None:
            return pandas.read_csv(self.param["data"])
        else:
            return pandas.read_csv(self.param["data"], nrows=nrows)
    
    def __str__(self) -> str:
        return f"Log({self.json_log_path})"
    
    def serialize(self) -> str:
        """
        序列化的时候忽略属性"other"
        """
        temp_json = self.param.copy()
        temp_json.pop("other")
        temp_json.pop("data")
        return json.dumps(temp_json)
    

# log1 = Log("./log/20231224063327_11cd96b4-ca81-4e25-ad9e-5aec0c3f3fa3.json")
# print(log1.param_names, log1["normId"])
# log1["data"]
# print(log1.serialize())
# log1.get_data()
# del log1

In [ ]:
import pandas
import os

NROWS = 200000
logs = list()
for root, dirs, files in os.walk("./log/"):
    for file in files:
        if file.endswith(".json"):
            logs.append(Log(os.path.join(root, file))) 
            print(os.path.join(root, file))
print(logs)

# 对logs按照norm的大小排序
logs = sorted(logs, key=lambda log:log["normId"])

# 限制读取行数为 100000 行
data = logs[0].get_data(NROWS)
data

In [ ]:
# sort logs by normId
logs = sorted(logs, key=lambda log:log["normId"])

print("logs_num:", len(logs))
filtered_logs = list()
for log in logs:
    if log["stepNum"] == 50000 and log['mu'] == 0.001 and log['p0'] == 0.5 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error" and log['gamma'] == 2:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 0.5:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
    # if log["stepNum"] == 100000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
        # 时间
        time_str = log['data'].split("/")[-1].split("_")[0]
        if int(time_str[:8]) >= 20240321:
            print(log.serialize())
            filtered_logs.append(log)
print("filtered_logs_num:", len(filtered_logs))

In [ ]:
logs = filtered_logs
end = NROWS
for log in logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    plt.figure(figsize=(11, 5))
    for i in range(1, 17):
        plt.plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=colors[i - 1])
    # 刻度间隔
    x_major_locator = plt.MultipleLocator(5000)
    y_major_locator = plt.MultipleLocator(0.1)
    ax = plt.gca()
    ax.xaxis.set_major_locator(x_major_locator)
    ax.yaxis.set_major_locator(y_major_locator)
    # 设置刻度
    margin = 0.02
    plt.xlim(-log["stepNum"]*margin, log["stepNum"]*(1+margin))
    plt.ylim(-1*margin*2, 1 + margin*2)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), ncol=8, fancybox=True, shadow=True)
    plt.xlabel('Time', fontsize=15)
    plt.ylabel('Donor-recipient Strategies', fontsize=15)
    plt.title("Social Norm Id: " + str(log["normId"]), fontsize=15)
    # plt.title("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    plt.tight_layout()
    plt.savefig('./temp_fig/longterm/' + 'd-r_norm' + str(log["normId"]) + '.png', dpi=600, bbox_inches='tight')

In [ ]:
logs = filtered_logs
end = NROWS
plt.figure(figsize=(10, 5))
idx = 0
for log in logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    i = 6
    label = "norm" + str(log["normId"])
    plt.plot(data.iloc[:end:4, 0], data.iloc[:end:4, i], label=label, color=colors[idx])
    idx+=1
    # 刻度间隔
    x_major_locator = plt.MultipleLocator(4000)
    y_major_locator = plt.MultipleLocator(0.1)
    ax = plt.gca()
    ax.xaxis.set_major_locator(x_major_locator)
    ax.yaxis.set_major_locator(y_major_locator)
    # 设置刻度
    margin = 0.02
    plt.xlim(-log["stepNum"]*margin, log["stepNum"]*(1+margin))
    plt.ylim(-1*margin*2, 1 + margin*2)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=8, fancybox=True, shadow=True)
    plt.xlabel('Time', fontsize=15)
    plt.ylabel('DISC-SR\nabundance', fontsize=15)
    # plt.title("Social Norm Id: " + str(log["normId"]), fontsize=15)
    # plt.title("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    plt.tight_layout()
    # plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
plt.savefig('./temp_fig/' + 'd-r_norm' + "16_norms" + '.png', dpi=600)

In [ ]:
for log in logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    plt.figure(figsize=(20, 4))
    fig, axes = plt.subplots(1,2,figsize=(20,6))
    fig.suptitle("Norm Id: " + str(log["normId"]), fontsize=15)
    fig.supxlabel('Time', fontsize=15)
    # 紧凑布局
    fig.tight_layout()
    # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    for i in range(17, 21):
        color_index = i - 17
        axes[0].plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=colors[color_index])
        axes[0].set_title("Donor")
        axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=4, fancybox=True, shadow=True)
        axes[0].set_ylabel('Strategies', fontsize=15)
        axes[1].plot(data.iloc[:end, 0], data.iloc[:end, i+4], label=data.columns.values[i+4], color=colors[color_index])
        axes[1].set_title("Recipient")
        axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=4, fancybox=True, shadow=True)
    plt.savefig('./temp_fig/' + 'd_r_norm' + str(log['normId']) + '.png', dpi=600)

In [ ]:
# 画第25列
for log in logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    # plt.figure(figsize=(20, 6))
    fig, axes = plt.subplots(1,2,figsize=(15, 4))
    # fig.subtitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    fig.suptitle("Norm Id: " + str(log["normId"]), fontsize=15)
    fig.supxlabel("Time")
    # fig.subplots_adjust(top=0.8)
    fig.tight_layout()
    color_index = 0
    for i in range(25, 27):
        index = i - 25
        if (data.columns.values[i] == 'good_rep'):
            # axes[index].set_xlabel(data.columns.values[0])
            axes[index].set_ylabel("Good Reputation")
        elif (data.columns.values[i] == 'cr'):
            axes[index].set_ylabel("Cooperation Rate")
        else: 
            raise "error"
        axes[index].plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=colors[color_index])
        color_index += 1
    # plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True)
    # plt.title()
    # plt.savefig('./norm10-c-ur.png', dpi=600)


In [ ]:
# 画第25列

fig = plt.figure(figsize=(10, 6))
color_index = 0
plt.xlabel("Time")
for log in filtered_logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    # plt.figure(figsize=(20, 6))
    # fig.subtitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    # fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    # fig.subplots_adjust(top=0.8)
    
    i = 26
    index = i - 25
    plt.ylabel("Good Reputation")
    plt.plot(data.iloc[:end, 0], data.iloc[:end, i], color=colors[color_index], label="norm" + str(log["normId"]))
    color_index += 1
    # plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True)
    # plt.title()
    # plt.savefig('./norm10-c-ur.png', dpi=600)

# 刻度间隔
x_major_locator = plt.MultipleLocator(5000)
y_major_locator = plt.MultipleLocator(0.1)
ax = plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
ax.yaxis.set_major_locator(y_major_locator)
margin = 0.02
plt.xlim(-log["stepNum"] * margin, log["stepNum"] * (1 + margin))
plt.ylim(-1 * margin * 2, 1 + margin * 2)
plt.xlabel("stepNum", fontsize=15)
plt.ylabel("Cooperation rate", fontsize=15)
plt.tight_layout()
plt.savefig('./cr_all_norm.png', dpi=600)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=8, fancybox=True, shadow=True)
plt.tight_layout()

In [ ]:
import pandas
import os

NROWS = 200000
logs = list()
for root, dirs, files in os.walk("./log/"):
    for file in files:
        if file.endswith(".json"):
            logs.append(Log(os.path.join(root, file))) 
            print(os.path.join(root, file))
print(logs)

# 对logs按照norm的大小排序
logs = sorted(logs, key=lambda log:log["normId"])

In [ ]:
# sort logs by normId
logs = sorted(logs, key=lambda log:log["normId"])

print("logs_num:", len(logs))
filtered_logs = list()
for log in logs:
    if log["stepNum"] == 50000 and log['mu'] == 0.001 and log['p0'] == 0.5 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 0.5:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
    # if log["stepNum"] == 100000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
        # 时间
        time_str = log['data'].split("/")[-1].split("_")[0]
        if int(time_str[:8]) >= 20240321:
            print(log.serialize())
            filtered_logs.append(log)
print("filtered_logs_num:", len(filtered_logs))

In [ ]:
gamma2logs_dict = dict()
for log in filtered_logs:
    if log["gamma"] not in gamma2logs_dict.keys():
        gamma2logs_dict[log["gamma"]] = list()
    gamma2logs_dict[log["gamma"]].append(log)

for key, ls in gamma2logs_dict.items():
    print(f"key: {key} len: {len(ls)}")
    output_str = ""
    for log in ls:
        output_str += str(log["normId"]) + " "
    print(output_str)

In [ ]:
import numpy as np
import matplotlib
matplotlib.rcParams['text.usetex'] = False

# 画第25列
# 计算一定区间内的平均合作率  以 norm 为 x 画出柱状图
range_start = 10000
range_end = 50000

fig = plt.figure(figsize=(10, 6))
N = 16
gamma2mean_cr = dict()
for key, logs in gamma2logs_dict.items():
    for log in logs:
        data = log.get_data(NROWS)
        # print(data.iloc[:, 26])
        # 取出 stepNum 在 range_start 和 range_end 之间的数据
        filtered_data = data[data['step'].between(range_start, range_end)]
        cr_series = filtered_data.iloc[:, 26]
        if key not in gamma2mean_cr.keys():
            gamma2mean_cr[key] = list()
        gamma2mean_cr[key].append(cr_series.mean())
    print(f"len: {len(gamma2mean_cr[key])}")


# 画n条并列柱状图
# 并列条数
bar_num = len(gamma2mean_cr.keys())
colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
width = 0.35
color_id = 0
bar_id = 0
for key, mean_cr in gamma2mean_cr.items():
    ind = np.arange(N) + width*bar_id
    print(f"key: {key} value: {mean_cr}")
    # plt.bar(ind, mean_cr, width, label='mean_cr', color=colors[color_id])
    print(color_id)
    b = 4
    gamma = int(key)
    # 开启 latex 渲染
    plt.bar(ind, mean_cr, width, label=r"$\frac{b}{\gamma}=" + f"{b/gamma}$", color=colors[color_id])
    color_id += 1
    bar_id += 1

# 刻度间隔
y_major_locator = plt.MultipleLocator(0.1)
ax = plt.gca()
ax.yaxis.set_major_locator(y_major_locator)
margin = 0.02
# plt.xlim(-log["stepNum"] * margin, log["stepNum"] * (1 + margin))
plt.xticks(range(0, N+1))


plt.ylim(-1 * margin * 2, 1 + margin * 2)
plt.xlabel("Norm id", fontsize=15)
plt.ylabel("Average cooperation rate", fontsize=15)
plt.tight_layout()
plt.savefig('./cr_all_norm.png', dpi=600)
# plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=8, fancybox=True, shadow=True)
plt.tight_layout()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.10), ncol=8, fancybox=True, shadow=True)
# plt.savefig('./temp_fig/' + 'cr_all_norm' + '.png', dpi=600)

In [ ]:
data[data['step'].between(0, 1000)]

In [ ]:
# 柱状图实例
import numpy as np
import matplotlib.pyplot as plt

N = 5
menMeans = (20, 35, 30, 35, 27)
womenMeans = (25, 32, 34, 20, 25)

ind = np.arange(N)  # the x locations for the groups
width = 0.35  # the width of the bars


fig, ax = plt.subplots()

rects1 = ax.bar(ind - width/2, menMeans, width, label='Men')
rects2 = ax.bar(ind + width/2, womenMeans, width, label='Women')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.set_xticks(ind)
ax.set_xticklabels(('G1', 'G2', 'G3', 'G4', 'G5'))
ax.legend()

plt.show()


In [ ]:
!jupyter nbconvert --to markdown visualize_json_longterm.ipynb

In [ ]:
# 用pandoc 转为 pdf
!pandoc visualize_json_longterm.md -o visualize_json_longterm.pdf --pdf-engine=xelatex  -V geometry:margin=1in 